<a href="https://colab.research.google.com/github/nitinsanatan/NLP_Basic_Tasks/blob/main/IMDB_Yelp_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

In [14]:
amazon_data = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
amazon_data.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [15]:
cols = ['Review','Sentiment']
amazon_data.columns = cols

In [16]:
yelp_data = pd.read_csv('yelp_labelled.txt', sep='\t', header=None)
yelp_data.columns = cols
yelp_data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [17]:
imdb_data = pd.read_csv('imdb_labelled.txt', sep='\t', header=None)
imdb_data.columns = cols
imdb_data.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [41]:
data = amazon_data.append([imdb_data,yelp_data], ignore_index=True)

In [43]:
data.shape

(2748, 2)

In [19]:
import string
punc = string.punctuation
punc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
import spacy

In [23]:
from spacy.lang.en.stop_words import STOP_WORDS
stopword = list(STOP_WORDS)
len(stopword)

326

In [24]:
nlp = spacy.load('en_core_web_sm')

In [37]:
def text_clean(sentence):
  doc = nlp(sentence)
  tokens = []
  for word in doc:
    if word.lemma_ != '-PRON-':
      text = word.lemma_.lower().strip()
    else:
      text = word.lower()
    tokens.append(text)
  #print(tokens)

  per_token = []
  for word in tokens:
    if word not in punc and word not in stopword:
      per_token.append(word)
  
  return per_token



In [29]:
text_clean("Heyy!! It's Barack Obama Speaking on call. Ok!")

['heyy', '!', '!', 'it', 'be', 'barack', 'obama', 'speak', 'on', 'call', '.', 'ok', '!']


['heyy', 'barack', 'obama', 'speak', 'ok']

In [30]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [46]:
tfidf = TfidfVectorizer(tokenizer=text_clean)

In [32]:
svm = LinearSVC()

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['Review'],data['Sentiment'], test_size=0.2)

In [45]:
X_train.shape, X_test.shape

((2198,), (550,))

In [47]:
model = Pipeline([('tfidf', tfidf), ('svm',svm)])

In [48]:
model.fit(X_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_clean at 0x7f36b8042050>)),
                ('svm', LinearSVC())])

In [52]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [50]:
pred = model.predict(X_test)

In [51]:
accuracy_score(pred, y_test)

0.7854545454545454

In [54]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.79      0.76      0.77       267
           1       0.78      0.81      0.80       283

    accuracy                           0.79       550
   macro avg       0.79      0.78      0.78       550
weighted avg       0.79      0.79      0.79       550



In [55]:
print(confusion_matrix(y_test, pred))

[[203  54]
 [ 64 229]]


In [57]:
print(model.predict(['It was a good movie']))

[1]
